# 히스토그램 변환  (Histogram modification)

## 영상의 명암비 조절

In [ ]:
# cv2.normalize(src, dst=None, alpha=None, beta=None, norm_type=None, dtype=None, mask=None) -> dst
# src: 입력영상
# dst: 결과영상
# alpha: 정규화 최소값 (예, 0)
# beta: 정규화 최댓값 (예, 155)
# norm_type: cv2.NORM_MINMAX
# dtype =  -1, src와 같은 type

In [34]:
import sys
import numpy as np
import cv2


src = cv2.imread('fig/manjang.jpg', cv2.IMREAD_REDUCED_GRAYSCALE_2)
# src = cv2.imread('fig/puppy_shrink.png', cv2.IMREAD_REDUCED_GRAYSCALE_2)

if src is None:
    print('Image load failed!')
    sys.exit()

# minMaxLoc(src[, mask]) -> minVal, maxVal, minLoc, maxLoc
smin, smax, _, _ = cv2.minMaxLoc(src)

dst_norm = cv2.normalize(src, None, 0, 255, cv2.NORM_MINMAX, -1)
# dst = np.clip(255*(src-smin)/(smax-smin) + 0, 0, 255).astype(np.uint8)

dst_equal = cv2.equalizeHist(src)


cv2.imshow('src', src)
cv2.imshow('dst_norm', dst_norm)
cv2.imshow('dst_equal', dst_equal)

cv2.waitKey()

cv2.destroyAllWindows()


## 칼라영상 히스토그램 평활화

In [ ]:
# equalizeHist(src, dst=None) -> dst
# src: 입력영상,gray scale 영상만 가능

In [31]:
import sys
import numpy as np
import cv2


# 컬러 영상의 히스토그램 평활화
src = cv2.imread('fig/spring_in_park.jpg', cv2.IMREAD_REDUCED_COLOR_2)

if src is None:
    print('Image load failed!')
    sys.exit()

src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(src_hsv) # list ret

# 밝기 성분에 대해서만 히스토그램 평활화 수행
v_equal = cv2.equalizeHist(v)
v_norm = cv2.normalize(v, None, 0, 255, cv2.NORM_MINMAX, -1)

dst_equal = cv2.merge((h, s, v_equal))
dst_norm = cv2.merge((h, s, v_norm))


dst_equal = cv2.cvtColor(dst_equal, cv2.COLOR_HSV2BGR)
dst_norm = cv2.cvtColor(dst_norm, cv2.COLOR_HSV2BGR)


cv2.imshow('src', src)
cv2.imshow('dst_equal', dst_equal)
cv2.imshow('dst_norm', dst_norm)

cv2.waitKey()

cv2.destroyAllWindows()

## 히스토그램 평활화를 통해서 개선할 수 있는 영상을 찾고 평활화 수행

## 특정 색상 영역 찾아 내기

In [ ]:
# inRange(src, lowerb, upperb[, dst]) -> dst
# src: 입력영상
# lowerb: 하한값 (0, 0, 0)
# upperb: 상한값 (255, 255, 255)

In [42]:
# src = cv2.imread('fig/palette.png')

# src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
# src_hsv[:,:,2] = src_hsv[:,:,2] - 100

# src = cv2.cvtColor(src_hsv, cv2.COLOR_HSV2BGR)
# cv2.imwrite('palette_dark.png', src)

# cv2.imshow('image', src)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [44]:
import sys
import numpy as np
import cv2

# src = cv2.imread('fig/palette.png')
src = cv2.imread('fig/palette_dark.png')


if src is None:
    print('Image load failed!')
    sys.exit()

src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

dst1 = cv2.inRange(src, (0, 128, 0), (100, 255, 100)) # b, g, r
dst2 = cv2.inRange(src_hsv, (40, 150, 0), (70, 255, 255)) # h, s, v

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)

cv2.waitKey()

cv2.destroyAllWindows()

In [1]:
## 트랙바를 이용해서 색상 찾기

import sys
import numpy as np
import cv2


src = cv2.imread('fig/palette_dark.png')

if src is None:
    print('Image load failed!')
    sys.exit()

src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

def on_trackbar(pos):
    hmin = cv2.getTrackbarPos('H_min', 'dst')
    hmax = cv2.getTrackbarPos('H_max', 'dst')
#     sat = cv2.getTrackbarPos('sat', 'dst')
    
    dst = cv2.inRange(src_hsv, (hmin, 150, 0), (hmax, 255, 255))
#     dst = cv2.inRange(src_hsv, (hmin, sat, 0), (hmax, 255, 255))
    cv2.imshow('dst', dst)


cv2.imshow('src', src)
cv2.namedWindow('dst')

cv2.createTrackbar('H_min', 'dst', 50, 179, on_trackbar)
cv2.createTrackbar('H_max', 'dst', 80, 179, on_trackbar)
# cv2.createTrackbar('sat', 'dst', 150, 255, call_trackbar)

on_trackbar(0)

cv2.waitKey()

cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:862: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


## 히스토그램 역투영

In [ ]:
# selectROI(windowName, img[, showCrosshair[, fromCenter]]) -> retval
    # retval:  x(왼쪽 상단), y(왼쪽상단), w (넓이), h(높이)

# calcBackProject(images, channels, hist, ranges, scale[, dst]) -> dst

In [2]:
import sys
import numpy as np
import cv2


src = cv2.imread('./fig/green.png', cv2.IMREAD_REDUCED_COLOR_2)

if src is None:
    print('image read failed')
    sys.exit()

x, y, w, h = cv2.selectROI(src)
src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb) 
crop = src_ycrcb[y:y+h, x:x+w]

# 0 번은 조명/밝기 정보, [1,2] = Cr, Cb channel
channels = [1, 2]
# histSize
cr_bins = 256 
cb_bins = 256
histSize = [cr_bins, cb_bins]

## range: [0, 256, 0, 256]
cr_range = [0, 256]
cb_range = [0, 256]
ranges = cr_range + cb_range

hist = cv2.calcHist([crop], channels, None, histSize, ranges)

# calcBackProject(images, channels, hist, ranges, scale[, dst]) -> dst
# dst: mask image
backproj = cv2.calcBackProject([src_ycrcb], channels, hist, ranges, 1)
dst = cv2.copyTo(src, backproj)


cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('hist', hist)
cv2.imshow('backproj', backproj)

cv2.waitKey()

cv2.destroyAllWindows()